In [ ]:
!pip install ultralytics

> *Detection YOLO for Face Detection: Use YOLO to detect faces in the scene.Face Recognition for Identification: After detection, pass the cropped face regions to a face recognition model*

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

import os

# **Data Visualization**

In [3]:
# import os
# import random
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from PIL import Image

# # Define paths
# train_images_dir = '/kaggle/input/attendence-dataset-for-dnn/Yolov8_label_images.v1i.yolov8/train/images'
# train_labels_dir = '/kaggle/input/attendence-dataset-for-dnn/Yolov8_label_images.v1i.yolov8/train/labels'

# # Select a random image and corresponding label file
# image_files = [f for f in os.listdir(train_images_dir) if f.endswith('.jpg')]
# random_image_file = random.choice(image_files)

# image_path = os.path.join(train_images_dir, random_image_file)
# label_path = os.path.join(train_labels_dir, random_image_file.replace('.jpg', '.txt'))

# # Load image
# image = Image.open(image_path)
# image_width, image_height = image.size

# # Load labels
# annotations = []
# if os.path.exists(label_path):
#     with open(label_path, 'r') as f:
#         for line in f:
#             parts = line.strip().split()
#             cls, x_center, y_center, width, height = map(float, parts)
#             annotations.append((cls, x_center, y_center, width, height))

# # Visualize the image with annotations
# fig, ax = plt.subplots(1, 1, figsize=(12, 8))
# ax.imshow(image)

# # Draw bounding boxes
# for cls, x_center, y_center, width, height in annotations:
#     # Convert normalized coordinates to absolute pixel values
#     abs_x = x_center * image_width
#     abs_y = y_center * image_height
#     abs_w = width * image_width
#     abs_h = height * image_height

#     # Top-left corner of the bounding box
#     top_left_x = abs_x - abs_w / 2
#     top_left_y = abs_y - abs_h / 2

#     # Draw rectangle
#     rect = patches.Rectangle(
#         (top_left_x, top_left_y), abs_w, abs_h,
#         linewidth=2, edgecolor='red', facecolor='none'
#     )
#     ax.add_patch(rect)

#     # Add label text
#     ax.text(
#         top_left_x, top_left_y - 10, f"Class: {int(cls)}",
#         color='red', fontsize=10, bbox=dict(facecolor='yellow', alpha=0.5)
#     )

# ax.axis('off')
# ax.set_title(f"Image: {random_image_file} with Annotations")
# plt.show()

# Detection Yolo v11 Model

*Object detection is a task that involves identifying the location and class of objects in an image or video stream.*



In [ ]:
model = YOLO("yolo11m.pt")


model.train(data='/kaggle/input/detection-dataset/detection dataset/data.yaml',
            epochs=50,
            imgsz=640)

In [23]:
model.save('yolov11_finetuned_dtn.pt')

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the fine-tuned model
model = YOLO('yolov11_finetuned_dtn.pt')

# Input image path
input_image_path = '/kaggle/input/attendence-dataset-for-dnn/Yolov8_label_images.v1i.yolov8/train/images/WhatsApp-Image-2024-12-06-at-12-19-02-PM-1-_jpeg.rf.105d4b1bbbc237f460826d2a18b318bd.jpg'  # Replace with your image path

# Perform inference with a high confidence threshold
results = model(input_image_path, conf=0.1)  # Set confidence threshold (e.g., 0.7)

# Visualize and filter results
result = results[0]  # Assuming 'results' is a list
high_conf_detections = [detection for detection in result.boxes if detection.conf > 0.1]

# Draw and save filtered detections
if high_conf_detections:
    annotated_image = result.plot()  # Visualize with detections
    plt.imshow(annotated_image)
    plt.axis('off')
    plt.show()

    # Save the image
    save_path = 'high_conf_results.jpg'
    cv2.imwrite(save_path, annotated_image)
    print(f"Filtered results saved to {save_path}")
else:
    print("No detections with confidence above the threshold.")


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the fine-tuned face detection model (replace with your face-specific model)
model = YOLO('yolov8n.pt')  # Ensure this model is trained to detect faces

# Input image path
input_image_path = '/kaggle/input/attendence-dataset-for-dnn/Yolov8_label_images.v1i.yolov8/train/images/WhatsApp-Image-2024-12-06-at-12-19-02-PM-1-_jpeg.rf.105d4b1bbbc237f460826d2a18b318bd.jpg'  # Replace with your image path

# Perform inference with a high confidence threshold
results = model(input_image_path, conf=0.6)  # Set confidence threshold

# Visualize and filter results
result = results[0]  # Assuming 'results' is a list

# Filter detections for faces only (class index for "face" is assumed to be 0; adjust if needed)
face_detections = [
    detection for detection in result.boxes if detection.conf > 0.6 and detection.cls == 0
]

# Draw and save filtered face detections
if face_detections:
    annotated_image = result.plot()  # Visualize detections
    plt.imshow(annotated_image)
    plt.axis('off')
    plt.show()

    # Save the image
    save_path = 'face_detections.jpg'
    cv2.imwrite(save_path, annotated_image)
    print(f"Face detections saved to {save_path}")
else:
    print("No face detections with confidence above the threshold.")


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the fine-tuned model
model = YOLO('yolov8_finetuned.pt')

# Input image path
input_image_path = '/kaggle/working/results_WhatsApp-Image-2024-12-14-at-11-49-17-PM-3-_jpeg.rf.9a136273f05e084ce5c78d8d95e6f94d.jpg'  # Replace with your image path

# Perform inference
results = model(input_image_path)

# Assuming 'results' is a list, access the first result object
result = results[0]

# Get the bounding boxes and labels
boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes (x1, y1, x2, y2)
labels = result.names  # Class names for the predictions

# Load the original image
image = cv2.imread(input_image_path)

# Iterate through the detected boxes and crop the regions with annotations
cropped_images = []
for i, box in enumerate(boxes):
    # Extract coordinates for the bounding box
    x1, y1, x2, y2 = map(int, box)
    
    # Crop the region of interest (ROI)
    cropped_image = image[y1:y2, x1:x2]
    
    # Convert cropped image to RGB for displaying in matplotlib
    cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    
    # Add the cropped image to the list
    cropped_images.append(cropped_image_rgb)
    
    # Optionally, display cropped images
    plt.imshow(cropped_image_rgb)
    plt.axis('off')  # Hide axes
    plt.show()

# Optionally save cropped images with unique names
for i, cropped_image in enumerate(cropped_images):
    cropped_image_path = f"cropped_{i}.jpg"
    cv2.imwrite(cropped_image_path, cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))
    print(f"Cropped image saved as {cropped_image_path}")


In [ ]:
!yolo task=classify mode=train model=yolov8m-cls.pt data='/kaggle/input/classification-dataset' epochs=10 imgsz=128

In [ ]:
model=YOLO("yolov8m-cls.pt")
model.train(data='/kaggle/input/classification-dataset',
            epochs=50,
            imgsz=640)

In [ ]:
model.save("yolo_finetuned_cls.pt")

In [ ]:
model = YOLO("/kaggle/working/yolo_finetuned_cls.pt")

In [ ]:
# Import necessary libraries
from ultralytics import YOLO

# Load the trained model
model = YOLO('/kaggle/working/yolo_finetuned_cls.pt')  # Path to the best classification model

# Perform prediction on a single image (classification task)
results = model.predict('/kaggle/working/cropped_0.jpg')

# Access the first result from the list
result = results[0]

# Display the classification result
result.show()  # Displays the image with the predicted class label

# Get the top predicted class and its confidence
predicted_class_index = result.probs.top1  # Index of the class with the highest probability
predicted_class = result.names[predicted_class_index]  # Get class label
confidence = result.probs.top1conf  # Confidence score for the prediction

# Print the classification results (class and confidence)
print("Predicted Class:", predicted_class)
print("Confidence:", confidence)